# 2-clean&filter

In [1]:
import pandas as pd

df = pd.read_csv("../data/extract_16.csv", low_memory=False, dtype={"ID_orateur": str})
df.shape

(337030, 32)

In [2]:
df = df[df["Code_grammaire"] == "PAROLE_GENERIQUE"]

In [3]:
df.shape

(184905, 32)

In [4]:
df.head()

,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,Code_grammaire,Code_style,Code_parole,Role_debat,Nom_orateur,Qualite_orateur,ID_orateur,stime,Texte,Source_extraction
3,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_1,NaN,Mme la présidente,NaN,719874,1007.69,La parole est à Mme la ministre de la transiti...,point
4,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,Mme Agnès Pannier-Runacher,ministre de la transition énergétique,759832,1041.01,"Après la promulgation, vendredi, de la loi rel...",point
6,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,Mme Agnès Pannier-Runacher,ministre,759832,1162.28,Comment se dire écologiste lorsqu’on ne vote p...,point
8,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,Mme Agnès Pannier-Runacher,ministre,759832,1173.04,…lorsqu’on vante le modèle allemand qui repose...,point
10,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,PAROLE_GENERIQUE,NORMAL,PAROLE_1_2,NaN,Mme Agnès Pannier-Runacher,ministre,759832,1210.25,Si nous voulons atteindre la neutralité carbon...,point


In [5]:
# Exclure les prises de parole de "Mme la présidente" et "M. le président"
# col Role_debat : pas bien identifié en fait
# plutôt partir sur Nom_orateur

df = df[~df["Nom_orateur"].str.strip().isin(["M. le président", "Mme la présidente"])]

In [6]:
df.shape

(129769, 32)

In [7]:
df["Texte"].str.len().describe()

count    129769.000000
mean        736.203816
std         930.668303
min           3.000000
25%         200.000000
50%         442.000000
75%         922.000000
max       21406.000000
Name: Texte, dtype: float64

In [8]:
df["len_dirtytext"] = df["Texte"].str.len()

In [9]:
df["Code_parole"].value_counts(dropna=False)

Code_parole
PAROLE_1_2       70643
NaN              38219
AVIS_COM_1_20    10951
AVIS_GVT_1_20     9954
AVIS_COM_1_10        1
PAROLE_1_1           1
Name: count, dtype: int64

In [10]:
df.groupby("Code_parole", dropna=False)["len_dirtytext"].describe()

,count,mean,std,min,25%,50%,75%,max
Code_parole,,,,,,,,
AVIS_COM_1_10,1.0,1278.000000,NaN,1278.0,1278.0,1278.0,1278.00,1278.0
AVIS_COM_1_20,10951.0,389.245457,416.340444,4.0,96.0,273.0,538.00,5395.0
AVIS_GVT_1_20,9954.0,418.768033,581.874887,4.0,20.0,212.0,581.75,6481.0
PAROLE_1_1,1.0,32.000000,NaN,32.0,32.0,32.0,32.00,32.0
PAROLE_1_2,70643.0,819.476325,1053.234730,3.0,221.0,464.0,1000.00,21406.0
NaN,38219.0,764.379314,823.602402,3.0,246.0,529.0,1008.00,16991.0


In [11]:
# voir ici pour aviser ce qu'il faut virer :
df[df["Code_parole"] == "AVIS_COM_1_20"].head()

,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,Code_style,Code_parole,Role_debat,Nom_orateur,Qualite_orateur,ID_orateur,stime,Texte,Source_extraction,len_dirtytext
326,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,NORMAL,AVIS_COM_1_20,NaN,Mme Maud Bregeon,rapporteure,795990,8834.43,Avis défavorable. On ne va pas débattre pendan...,point,237
368,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,NORMAL,AVIS_COM_1_20,NaN,Mme Maud Bregeon,rapporteure,795990,10235.34,Je ne vous surprendrai pas : avis défavorable....,point,1048
433,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,NORMAL,AVIS_COM_1_20,NaN,Mme Maud Bregeon,rapporteure,795990,11406.98,Je ne sais pas si les énergies renouvelables f...,point,350
436,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,NORMAL,AVIS_COM_1_20,NaN,Mme Maud Bregeon,rapporteure,795990,11418.63,"Vous êtes aussi très nombreux, chers collègues...",point,361
454,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,NORMAL,AVIS_COM_1_20,NaN,Mme Maud Bregeon,rapporteure,795990,11788.01,Avis défavorable.,point,17


## Match députés

In [12]:
df_deputes = pd.read_csv("../data/raw/id-dep/deputes-historique(datan-datagouv).csv")

In [13]:
df_deputes

,id,legislatureLast,civ,nom,prenom,villeNaissance,naissance,age,groupe,groupeAbrev,...,facebook,website,nombreMandats,experienceDepute,scoreParticipation,scoreParticipationSpecialite,scoreLoyaute,scoreMajorite,active,dateMaj
0,PA1001,14,M.,Daubresse,Marc-Philippe,Lille,1953-08-01,71,Les Républicains,LES-REP,...,NaN,NaN,3,11 ans,0.91,NaN,1.000,0.109,0,2025-06-24
1,PA1008,17,M.,David,Alain,Libourne,1949-06-02,76,Socialistes et apparentés,SOC,...,AlainDavidAssembleeNationale,alain-david.com,3,8 ans,0.05,NaN,0.961,0.000,1,2025-06-24
2,PA1009,12,Mme,David,Martine,Loches,1952-12-19,72,Socialiste,SOC,...,NaN,NaN,1,5 ans,NaN,NaN,NaN,NaN,0,2025-06-24
3,PA1012,15,M.,de la Verpillière,Charles,Bourg-en-Bresse,1954-05-31,71,Les Républicains,LR,...,charlesdelaverpilliere,www.charlesdelaverpilliere.com,3,15 ans,0.12,0.26,0.950,0.247,0,2025-06-24
4,PA1020,14,M.,Debré,Bernard,Toulouse,1944-09-30,80,Les Républicains,LES-REP,...,NaN,www.bernarddebre.fr,3,13 ans,0.96,NaN,0.851,0.140,0,2025-06-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2081,PA951,14,M.,Couve,Jean-Michel,Le Muy,1940-01-03,85,Les Républicains,LES-REP,...,NaN,NaN,3,15 ans,0.94,NaN,0.959,0.102,0,2025-06-24
2082,PA954,12,M.,Cova,Charles,Bourg-en-Bresse,1931-12-09,93,Union pour un Mouvement Populaire,UMP,...,NaN,www.charlescova.com,1,5 ans,NaN,NaN,NaN,NaN,0,2025-06-24
2083,PA966,13,M.,Cuq,Henri,Toulouse,1942-03-12,83,Union pour un Mouvement Populaire,UMP,...,NaN,NaN,1,3 ans,NaN,NaN,NaN,NaN,0,2025-06-24
2084,PA998,15,M.,Dassault,Olivier,Boulogne-Billancourt,1951-06-01,74,Les Républicains,LR,...,olivier.dassault,www.olivier-dassault.fr/,4,19 ans,0.03,0.12,0.893,0.427,0,2025-06-24


In [14]:
df["ID_orateur"] = "PA" + df["ID_orateur"]

In [15]:
df.shape

(129769, 33)

In [16]:
df = df.join(df_deputes.set_index("id"), on="ID_orateur", how="left", rsuffix="_dep")

In [17]:
df.shape

(129769, 59)

In [18]:
df.head()

,UID,SeanceRef,SessionRef,DateSeance,DateSeanceJour,NumSeanceJour,NumSeance,TypeAssemblee,Legislature,Session,...,facebook,website,nombreMandats,experienceDepute,scoreParticipation,scoreParticipationSpecialite,scoreLoyaute,scoreMajorite,active,dateMaj
4,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,AgnesPannierR,NaN,1.0,4 mois,0.0,NaN,NaN,NaN,0.0,2025-06-24
6,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,AgnesPannierR,NaN,1.0,4 mois,0.0,NaN,NaN,NaN,0.0,2025-06-24
8,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,AgnesPannierR,NaN,1.0,4 mois,0.0,NaN,NaN,NaN,0.0,2025-06-24
10,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,AgnesPannierR,NaN,1.0,4 mois,0.0,NaN,NaN,NaN,0.0,2025-06-24
12,CRSANR5L16S2023O1N173,RUANR5L16S2023IDS26877,SCR5A2023O1,20230313160000000,lundi 13 mars 2023,1,173,AN,16,Session ordinaire 2022-2023,...,AgnesPannierR,NaN,1.0,4 mois,0.0,NaN,NaN,NaN,0.0,2025-06-24


## Export

In [19]:
df.shape

(129769, 59)

In [20]:
# ICI ÇA MERDAIT ENTRE LA SORTIE ECRITURE ET LA LECTURE/IMPORT DU DF
# Resolu avec quotes
# Voir pourquoi et où est le pb quand même dans les cols du join

In [21]:
import csv  # pour résoudre le soucis d'écart. Checker

df.to_csv(
    "../data/data_cleaning.csv",
    index=False,
    quoting=csv.QUOTE_ALL,  # a permis de résoudre le soucis d'écart. Checker
)

In [22]:
df_test = pd.read_csv(
    "../data/data_cleaning.csv", low_memory=False, dtype={"ID_orateur": str}
)

In [23]:
df_test.shape

(129769, 59)